## importing packages

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Scraping Wiki and extracting the table by finding table class in the html

extracting the data from table

In [2]:
#Scraping Wikipedia for the raw data
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url, 'lxml')
table = soup.find('table', {'class':'wikitable sortable'})

## Extracting the data from within all the table classes and appending them into lists

headers_result -- the 3 extracted table headers from the 'th' tags |
data_result -- the values extracted from the 'td' tags

In [3]:
#Extracting the Table Headers
headers = table.findAll('th')
headers_result = []
for i in range(len(headers)):
    s = str(headers[i])
    start = s.find('<th>') + len('<th>')
    end = s.find('</th>')
    headers_result.append(s[start:end-1])
    

#Extracting the data from the table
data = table.findAll('td')
data_result = []
for i in range(len(data)):
    s = str(data[i])
    start = s.find('<td>') + len('<td>')
    end = s.find('</td>')
    data_result.append(s[start:end-1])

## Creating the Dataframe for the raw data
### Conditional loop to not process any bourough == not assigned

In [4]:
#Creating the DataFrame and populating
df = pd.DataFrame(columns = headers_result) 
num_cols = len(headers_result)
num_rows = int(len(data_result)/num_cols)

i = 0 

for row_num in range(num_rows):
    pc = data_result[i]
    borough = data_result[i+1]
    neigh = data_result[i+2]
    
    #removing not assigned borough
    if borough=='Not assigned':
        i+=3
        next
    else:
        df.loc[row_num] = [pc, borough, neigh]
        i+=3

df.reset_index(drop=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## Seperating the Neighborhoods with ","

In [6]:
df['Neighborhood'].replace('/', ',', regex=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## If neighborhood is not assigned, it will be allocated the Borough name

In [7]:
for i in range(len(df)):
    if df['Neighborhood'][i] == 'Not assigned':
        df['Neighborhood'][i] = df['Borough'][i]

In [8]:
df.shape

(103, 3)